# Read in data

In [1]:
HCP=read.table('/dagher/dagher11/filip/HCP_beh//RESTRICTED_uku_7_10_2017_9_43_31.csv',sep=',',header=T)
HCP_u=read.table('/dagher/dagher11/filip/HCP_beh//unrestricted_uku_7_10_2017_9_42_13.csv',sep=',',header=T)
QC=read.table('/dagher/dagher11/filip/HCP_QC/HCP/QC_sbatch/HCP.csv',sep=',',header=T)
QC=QC[, 1:2]
colnames(QC)=c('Subject','QC')

# Merge datasets

In [2]:
All_data=merge(HCP,HCP_u, by='Subject')
All_data=merge(All_data,QC, by='Subject', all.x=T)

# Create initial FSGD

In [3]:
FSGD=data.frame('Variables'='Input', 'Subject'=All_data$Subject, 'Class'='main', 'BMI'=All_data$BMI, 
                'Sex'=All_data$Gender, 'Age'=All_data$Age_in_Yrs, 'QC'=All_data$QC, 'FID'=All_data$Family_ID,
                'Zyg'=All_data$ZygositySR, 'Ethnicity'=All_data$Ethnicity, 'Edu'=All_data$SSAGA_Educ,
                'Income'=All_data$SSAGA_Income, 'Hypothyroid'=All_data$Hypothyroidism, 
                'Hyperthyroid'=All_data$Hyperthyroidism, 'Endocrine'=All_data$OtherEndocrn_Prob)

FSGD[is.na(FSGD)] <- 0 # This is fine because I exclude participants who have 0 in QC and 
                        # leave participants who have 1 in diseases


# Exclude participants who did not pass CT QC

In [4]:
FSGD=FSGD[FSGD$QC==1,]
FSGD$QC=NULL

## Exclude one of each twin pair

In [5]:
FSGD_nt=FSGD[FSGD$Zyg=='NotTwin',]
FSGD_t=FSGD[(FSGD$Zyg=='MZ' | FSGD$Zyg=='NotMZ'),]
FSGD_t=FSGD_t[!duplicated(FSGD_t$FID),]
FSGD=rbind(FSGD_nt,FSGD_t)

# Exclude people with endocrine issues

In [6]:
FSGD=FSGD[FSGD$Hypothyroid==0,]
FSGD=FSGD[FSGD$Hyperthyroid==0,]
FSGD=FSGD[FSGD$Endocrine==0,]

# Remove unnecessary columns

In [7]:
FSGD$FID=NULL
FSGD$Zyg=NULL
FSGD$Hypothyroid=NULL
FSGD$Hyperthyroid=NULL
FSGD$Endocrine=NULL
FSGD$Ethnicity=NULL
FSGD$Sex=as.numeric(FSGD$Sex)

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [11]:
length(FSGD$Age)

[1] 814

# Save FSGD

In [246]:
write.table(FSGD, '/dagher/dagher11/filip/Obesity_maps/data/analysis_noethn.fsgd', sep='\t', quote=F, row.names=F)